In [1]:
import pandas as pd
import numpy as np

def read_excel(file_name):
    df = pd.read_excel(file_name)
    return df

def read_txt(file_name):
    file = open(file_name)
    lines = file.readlines()
    return(lines[0])

/tmp/ipykernel_171206/787217452.py:1: DeprecationWarning: 
Pyarrow will become a required dependency of pandas in the next major release of pandas (pandas 3.0),
(to allow more performant data types, such as the Arrow string type, and better interoperability with other libraries)
but was not found to be installed on your system.
If this would cause problems for you,
please provide us feedback at https://github.com/pandas-dev/pandas/issues/54466
        
  import pandas as pd


In [2]:
import os
import glob

def get_files(subfolder, extension):
    dir = f"{os.getcwd()}/content/{subfolder}/"
    tables = glob.glob(f"{dir}*.{extension}")
    return tables

In [3]:
class Analizer:
    def __init__(self, boundary):
        self.results = get_files(subfolder="results", extension="xlsx")
        self.results_df = pd.DataFrame()
        self.boundary = boundary
    
    def has_minimum_requirements(self, df, sort_by="r2"):
        sorted_df = df.sort_values(by=sort_by, ascending=False)
        top_r2 = sorted_df.head(1)[sort_by].values[0]
        if top_r2 < self.boundary:
            return False
        return True
    
    def concatenate_df(self, df, architecture):
        if self.has_minimum_requirements(df):
            df['Architecture'] = architecture
            df = df.rename(columns={'Unnamed: 0': 'model'})
            self.results_df = pd.concat([self.results_df, df], ignore_index=True) 

    def create_results_df(self):
        for file in self.results:
            df = read_excel(file)
            architecture = read_txt(file.replace(".xlsx", ".txt"))
            self.concatenate_df(df, architecture)
        self.results_df = self.results_df.sort_values(by="r2", ascending=False, ignore_index=True)

    def discard_below_average(self, sort_by):
        column_mean = self.results_df[sort_by].mean()      
        self.results_df = self.results_df[self.results_df[sort_by] >= column_mean]
    
    def discard_high_standard_deviation(self):
        r2_val, r2_test = self.results_df['r2_val'], self.results_df['r2_test']
        std_devs = np.abs(r2_val - r2_test)
        mean_std_dev = std_devs.mean()
        self.results_df = self.results_df[std_devs < mean_std_dev]

    def clean_folder(self, subfolder, extension, remove_last=True):
        files = get_files(subfolder, extension)
        models = self.results_df["model"]
        if (remove_last):
            models = models.apply(lambda x: '_'.join(x.rsplit('_', 1)[:-1]))
        for file in files:
            file_name = os.path.basename(file).split('.')[0]
            file_parts = file_name.split('_')            
            dataset_model = f"model_{file_parts[1]}_{file_parts[2]}" 
            if (remove_last == False):
                dataset_model = (f"{dataset_model}_{file_parts[3]}")
            if dataset_model not in models.values:
                os.remove(file)   
        
    def Analize(self):
        self.create_results_df()
        # self.discard_below_average(sort_by="r2")
        # self.discard_below_average(sort_by="r2_vt")
        # self.discard_high_standard_deviation()
        self.results_df.sort_values(by="mse")
        self.results_df.to_excel(f"./results/better_results.xlsx", index=True)
        display(self.results_df)


In [4]:
analize = Analizer(0.4)
analize.Analize()
# analize.clean_folder(subfolder="dataset", extension="pkl")
# analize.clean_folder(subfolder="results", extension="xlsx")
# analize.clean_folder(subfolder="results", extension="txt")
# analize.clean_folder(subfolder="models", extension="keras", remove_last=False)



,model,r2,r2_sup,r2_test,r2_val,mse,mse_sup,mse_test,mse_val,mape,rmse,r2_adj,rsd,aic,bic,Architecture
0,model_1_41_0,0.999764,0.700605,0.635294,0.999723,0.000109,0.177734,0.255484,0.000160,0.005212,0.010439,0.999444,0.010532,148.248750,326.101649,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
1,model_1_24_2,0.999763,0.698711,0.635364,0.999757,0.000110,0.178858,0.255435,0.000132,0.005221,0.010471,0.999441,0.010564,148.236433,326.089332,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
2,model_1_24_3,0.999762,0.699285,0.636662,0.999752,0.000110,0.178518,0.254525,0.000135,0.005218,0.010488,0.999439,0.010581,148.230212,326.083112,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
3,model_1_40_9,0.999761,0.701001,0.635075,0.999717,0.000110,0.177499,0.255637,0.000142,0.005294,0.010495,0.999438,0.010588,148.227367,326.080266,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
4,model_1_40_8,0.999761,0.701006,0.635075,0.999717,0.000110,0.177496,0.255637,0.000141,0.005297,0.010501,0.999438,0.010594,148.225256,326.078155,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
495,model_1_0_4,0.602279,0.529281,0.589067,0.585699,0.183624,0.279439,0.287867,0.172285,0.242787,0.428514,0.063697,0.432323,133.389731,311.242630,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
496,model_1_0_3,0.497372,0.443160,0.522313,0.443805,0.232058,0.330565,0.334630,0.231291,0.285814,0.481724,-0.183271,0.486007,132.921532,310.774431,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
497,model_1_0_2,0.430801,0.342780,0.389416,0.378891,0.262794,0.390154,0.427726,0.258285,0.314206,0.512634,-0.339990,0.517191,132.672773,310.525672,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
498,model_1_0_1,0.318625,0.237709,0.290279,0.276220,0.314584,0.452529,0.497174,0.300979,0.339030,0.560878,-0.604070,0.565863,132.313009,310.165908,"Hidden Size=[12, 2], regularizer=0.02, learnin..."
